#### Capstone Project
Arthur: Wilson Lau
Date: 2015 November

### Capstone Milestone Report

## Introduction
The problem I want to tackle in my capstone project is to detect investment opportunities for cash flow real estate property.  

## The Problem
Real estate investment is a passive income opportunity for investors to accumulate return without active management.  Huge long term appreciation is very much possible for real estate.  However, appreciation often happens in short period of time and the appreciation is heavily affected local market condition.  It is difficult for out-of-the-market investors to detect the trend and acquire properties before price goes up.  My capstone project is trying to help investors to detect real estate investment opportunities and be able to predict which zip code will experience long term appreciation in the near future. 

My project is divided into two stages:

# Stage One
- the first stage is to use anomaly detection technique to discover which zipcodes have success sprike of upward price appreciation. 
- The Anomaly detection is compared new monthly updated home price with historical price as well as comparing with the home price of the surrending areas based on neighborhoods, city, menpolian area, and state.

# Stage Two
- Based on the result of stage one, more data will be collected for the zipcodes that have sudden increase of home price.  
- Those new dataset will be used to figure out what feature contributes to the price appreciation and a predictive model will be created using those features. 

## Dateset
Dataset used is downloaded from Quandl's Zillow data.  This dataset is a monthly time series recording the change of home price. 

# What important fields and information does the data set have?
The important columns for this dataset is time(date end of the month), home price and zipcode. 



# What are its limitations i.e. what are some questions that you cannot answer with this data set?
This dataset actually have 22 columns.  For example, it has columns for different home size e.g. 1 bedroom, 2 bedroom, 3 bedroom, price per square foot, average listing price, etc...  Since it seems that most of these columns are highly corelate to home price, they may be very help for predicting home price. 

# What kind of cleaning and wrangling did you need to do?
Quandl's API is very powerful and very flexible.  At the same time, it is a bit complicated to pull the home price.  One way to pull home price is to pass zipcode as input parameter, and Quandl will return the historical price data with all the columns related to that zipcode.  So, to get the data, I will need to use a zipcode dataset that contains all the zipcode of the US and then use the Quandl API to pull data one zipcode at a time. 

# Are there other datasets you can find, use and combine with, to answer the questions that matter?
To order to group home price data by city, state, etc.., I will need to find other dataset that relates individual zipcodes with their city name and state name.  I will need to merge the dataset together in order to find out which zipcode belongs to which city or state. 

## Preliminary Findings and Hypothesis test.
.....






In [1]:
###load home price data into dataframe
import numpy as np
import pandas as pd
import pickle
from scipy import stats

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

df3 = pickle.load( open( ".\DataPfiles\90621_to_91024.p", "rb" ) )
df = pickle.load( open( ".\DataPfiles\93063_to_93524.p", "rb" ) ) 

t1 = df.iloc[:,1] 
t2 = df3.iloc[:,0]
print "mean is % 4.3F and sd is % 4.3F " % (t1.mean(),t1.std())


###pivot All Homes prirce to row and grouped by zipcode
df = df.reset_index()
dfAllHomes = df.pivot(index='index', columns='ZipCode', values='All Homes')

##loop through each zipcode column
for x in dfAllHomes.columns:
    if sum(dfAllHomes[x].isnull()) > 0:
#         print x
        dfAllHomes.drop(x,axis=1)
    dfAllHomes[x] = movingaverage(dfAllHomes[x],3)  #don't use the first and last record of the moving average of 3..
#     print dfAllHomes[x].ix(1,:)  # try to remove the first record for each zip code..
#     dfAllHomes[x].iloc(-1,:).drop()
    dfAllHomes[x] = (dfAllHomes[x] - dfAllHomes[x].mean()) / dfAllHomes[x].std()  #calculate z score for All Homes price each month

print dfAllHomes.head()


# add a new column for moving average of All Homes price


p = df11.iloc[:,0] #ALl Homes price
pz = df11.iloc[:,-1] 


window_size = 3.0 #set the number of sample to gathering centered in the middle
movingave = lambda x: np.convolve(x, np.ones(int(window_size))/float(window_size), 'same')
transformed = df11.groupby('ZipCode')
transformed['All Homes'].transform(movingave)
# df11.info()

# df11 = df11[df11['All Homes'].isnull()]
grouped = df11[['All Homes','ZipCode']].groupby('ZipCode')
b = pd.DataFrame()
newdf = pd.DataFrame()
for name,group in grouped:
    g = group.copy()
#     print group.shape
#     print "size: % 3.2F" % movingaverage(group['All Homes'],3).size
#     if sum(g.isnull()) < 0:
#         g['Moving Ave'] = movingaverage(group['All Homes'],3)

#     print g.shape
#     print g.head()

    #     if(newdf.isnull):
#         newdf = g
#     newdf = newdf.append(g)
#     b = group['All Homes']
#     a = movingaverage(group['All Homes'],3)
    
#     group["MovingAve"] = np.convolve(group['All Homes'], np.ones(int(window_size))/float(window_size), 'same')
#     newdf = newdf.append(group)
# print b
# newdf
# newdf.info()
    
    


# df11['Moving Ave'] = movingaverage(df11.iloc[:,0], 3)  #para#1 is All Homes price; para#2 is numer of sample to gather centered in the middle

# df11.head()

mean is  240927.088 and sd is  169428.242 
ZipCode        93063     93065     93066  93067     93101     93103     93105  \
index                                                                           
1996-04-30 -2.047494 -2.075972 -2.258279    NaN -2.251027 -2.272345 -2.347812   
1996-05-31 -1.556610 -1.581733 -1.743935    NaN -1.890267 -1.870046 -1.937374   
1996-06-30 -1.558234 -1.582007 -1.767827    NaN -1.896817 -1.886940 -1.932706   
1996-07-31 -1.555257 -1.582280 -1.781183    NaN -1.898455 -1.899215 -1.922568   
1996-08-31 -1.551740 -1.580913 -1.778660    NaN -1.895343 -1.899215 -1.909229   

ZipCode     93108     93109     93110  ...       93505     93510  93512  \
index                                  ...                                
1996-04-30    NaN -2.287223 -2.250942  ...   -1.166732 -1.798483    NaN   
1996-05-31    NaN -1.908221 -1.828507  ...   -0.690530 -1.211365    NaN   
1996-06-30    NaN -1.908841 -1.840526  ...   -0.699607 -1.247532    NaN   
1996-07-31    

NameError: name 'df11' is not defined

In [33]:
p = df11.iloc[:,0]  #ALl Homes price
pz = df11.iloc[:,-1] 
print pz

1996-04-30    93063
1996-05-31    93063
1996-06-30    93063
1996-07-31    93063
1996-08-31    93063
1996-09-30    93063
1996-10-31    93063
1996-11-30    93063
1996-12-31    93063
1997-01-31    93063
1997-02-28    93063
1997-03-31    93063
1997-04-30    93063
1997-05-31    93063
1997-06-30    93063
1997-07-31    93063
1997-08-31    93063
1997-09-30    93063
1997-10-31    93063
1997-11-30    93063
1997-12-31    93063
1998-01-31    93063
1998-02-28    93063
1998-03-31    93063
1998-04-30    93063
1998-05-31    93063
1998-06-30    93063
1998-07-31    93063
1998-08-31    93063
1998-09-30    93063
              ...  
2011-01-31    93523
2011-02-28    93523
2011-03-31    93523
2011-04-30    93523
2011-05-31    93523
2011-06-30    93523
2011-07-31    93523
2011-08-31    93523
2011-09-30    93523
2011-10-31    93523
2011-11-30    93523
2011-12-31    93523
2012-01-31    93523
2012-02-29    93523
2012-03-31    93523
2012-04-30    93523
2012-05-31    93523
2012-06-30    93523
2012-07-31    93523


In [53]:
from pylab import plot, ylim, xlim, show, xlabel, ylabel
from numpy import linspace, loadtxt
import numpy as np

r=3.0

x = p.head()
y = pz

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

# plot(x,y)
# xlim(0,1000)

x_av = movingaverage(x, r)
# plot(x_av, y)

# xlabel("Months since Jan 1749.")
# ylabel("No. of Sun spots")
# show()
print x_av


[ 121333.33333333  181800.          181600.          181966.66666667
  121500.        ]


index
1996-04-30   -1.562054
1996-05-31   -1.570189
1996-06-30   -1.571003
1996-07-31   -1.566935
1996-08-31   -1.561240
1996-09-30   -1.560427
1996-10-31   -1.564495
1996-11-30   -1.566935
1996-12-31   -1.568562
1997-01-31   -1.570189
1997-02-28   -1.573444
1997-03-31   -1.578325
1997-04-30   -1.583206
1997-05-31   -1.584020
1997-06-30   -1.584020
1997-07-31   -1.585647
1997-08-31   -1.586461
1997-09-30   -1.584020
1997-10-31   -1.577511
1997-11-30   -1.565308
1997-12-31   -1.549850
1998-01-31   -1.531952
1998-02-28   -1.513240
1998-03-31   -1.493715
1998-04-30   -1.475817
1998-05-31   -1.461986
1998-06-30   -1.455478
1998-07-31   -1.450597
1998-08-31   -1.439207
1998-09-30   -1.422936
                ...   
2013-01-31    0.151297
2013-02-28    0.182212
2013-03-31    0.231839
2013-04-30    0.286347
2013-05-31    0.339229
2013-06-30    0.395364
2013-07-31    0.451499
2013-08-31    0.492991
2013-09-30    0.534482
2013-10-31    0.588991
2013-11-30    0.648380
2013-12-31    0.687431
2014-